In [ ]:
from metaflow import Metaflow, Flow
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.io as pio

def get_boxplot(args, filename="Boxplot.pdf", title=""):

    data = []
    for arg in args:

        trace = go.Box(
            y=arg['y'],
            name = arg['name']
        )
        data.append(trace)

    layout = go.Layout(
        title = title
    )

    fig = go.Figure(data=data,layout=layout)
    pio.write_image(fig, filename)


def get_barplot(args, filename="Barchart.pdf", title = ""):

    data = []

    for arg in args:

        trace = go.Bar(
            y = arg['y'],
            x = arg.get('name',np.arange(0,len(arg['y']))),
            name = arg.get('trace   ', 'trace')
        )

        data.append(trace)


    layout = go.Layout(
        #barmode = 'overlay',
        title = title
    )

    fig = go.Figure(data=data,layout=layout)
    pio.write_image(fig, filename)


def get_grouped_barplot(args, filename="Grouped_barplot.pdf", title="Comparison of per-call runtimes"):

    data=[go.Bar(name=arg['name'], x=arg['x'], y=arg['y']) for arg in args]

    print("Data ",data)

    fig = go.Figure(data=[go.Bar(name=arg['name'], x=arg['x'], y=arg['y']) for arg in args],
                    layout=go.Layout(title=title))
    # Change the bar mode
    fig.update_layout(barmode='group')
    pio.write_image(fig, filename)


def get_lineplot(args, filename="Linechart.pdf", title="", xaxis_title="", yaxis_title=""):

        data = []

        for arg in args:

            trace = go.Scatter(
                x = arg.get('x',np.arange(0,len(arg['y']))),
                y = arg['y'],
                name = arg.get('name',None),
                line = { 'color': arg.get('color',None) }
            )
            data.append(trace)

        layout = go.Layout(
            title = title,
            xaxis = { 'title': xaxis_title },
            yaxis = { 'title': yaxis_title }
        )

        fig = go.Figure(data=data,layout=layout)
        pio.write_image(fig, filename)


def get_time_per_tolerance(runs):

    def find_steps(convergence, tol_list):
        conv_array = np.array(convergence)
        res = [np.argmin(abs(conv_array - tol_list_elem)) for tol_list_elem in tol_list]
        return(res)

    for run in runs:
        steps = list(run.steps())
        a = steps[2]
        alist = find_steps(a.task.data.convergence, [0.1,0.05,0.01,0.005,0.001])
        b = steps[4]
        blist = find_steps(b.task.data.convergence, [0.1,0.05,0.01,0.005,0.001])
        c = steps[6]
        clist = find_steps(c.task.data.convergence, [0.1,0.05,0.01,0.005,0.001])
        d = steps[8]
        dlist = find_steps(d.task.data.convergence, [0.1,0.05,0.01,0.005,0.001])

    print(alist)
    print(blist)
    print(clist)
    print(dlist)
    args_linechart = [{'name': a.task.data.title, 'x': [0.1,0.05,0.01,0.005,0.001], 'y': np.array(alist) * a.task.data.time_per_call},
                      {'name': b.task.data.title, 'x': [0.1,0.05,0.01,0.005,0.001], 'y': np.array(blist) * b.task.data.time_per_call},
                      {'name': c.task.data.title, 'x': [0.1,0.05,0.01,0.005,0.001], 'y': np.array(clist) * c.task.data.time_per_call},
                      {'name': d.task.data.title, 'x': [0.1,0.05,0.01,0.005,0.001], 'y': np.array(dlist) * d.task.data.time_per_call}
                    ]

    get_lineplot(args_linechart, "Time_per_convergence.pdf", "Number of steps as a function of desired convergence tolerance")


def get_runtime_comparisons(runs):

    alist = []
    blist = []
    clist = []
    dlist = []
    for run in runs:
        steps = list(run.steps())
        a = steps[2]
        b = steps[4]
        c = steps[6]
        d = steps[8]

        alist.append(a.task.data.time_per_call)
        blist.append(b.task.data.time_per_call)
        clist.append(c.task.data.time_per_call)
        dlist.append(d.task.data.time_per_call)

    alist_mean = sum(alist) / len(alist)
    blist_mean = sum(blist) / len(blist)
    clist_mean = sum(clist) / len(clist)
    dlist_mean = sum(dlist) / len(dlist)

    args_boxplot = [{'name': a.task.data.title, 'y': alist},
            {'name': b.task.data.title, 'y': blist},
            {'name': c.task.data.title, 'y': clist},
            {'name': d.task.data.title, 'y': dlist}
            ]

    get_boxplot(args_boxplot, title="Comparison of per-call runtimes")

    args_barchart = [{'name': ['Large data', 'Animal data'], 'y': [alist_mean/ blist_mean, clist_mean/ dlist_mean] }]

    get_barplot(args_barchart, "Barchart.pdf", title="Average speedup of analytical gradients over finite differences")

    print([alist_mean, blist_mean])
    args_grouped_barplot = [{'name': 'Animal', 'x':[a.task.data.title, b.task.data.title], 'y': [alist_mean, blist_mean]},
            {'name': 'Big data', 'x': [c.task.data.title, d.task.data.title], 'y': [clist_mean, dlist_mean]}
            ]

    get_grouped_barplot(args_grouped_barplot, title="Comparison of runtimes for analytical and finite difference approaches")


def list_steps(run):

    def print_metrics(step):
        print("-------------------------------------------")
        print("Title ",step.task.data.title)
        print("Function calls ",step.task.data.fn_count)
        print("Time elapsed ",step.task.data.time_elapsed)
        print("Time per call",step.task.data.time_per_call)
        print("Residual ",step.task.data.nlopt_residual)

    print('--------------------------- Starting run ---------------------------', run.id, run.finished_at)
    steps = list(run.steps())
    a_finite = steps[4]
    a_svd = steps[2]
    b_finite = steps[6]
    b_svd = steps[8]

    print_metrics(a_finite)
    print_metrics(a_svd)
    print_metrics(b_finite)
    print_metrics(b_svd)


def get_convergence(runs):

    for run in runs:
        steps = list(run.steps())
        a = steps[2]
        b = steps[4]
        c = steps[6]
        d = steps[8]


    args_linechart = [{'name': a.task.data.title, 'y': a.task.data.convergence},
                      {'name': b.task.data.title, 'y': b.task.data.convergence},
                      {'name': c.task.data.title, 'y': c.task.data.convergence},
                      {'name': d.task.data.title, 'y': d.task.data.convergence}
                    ]

    get_lineplot(args_linechart)



def list_flows(mf):
    print(mf.flows)
    flows =list(mf.flows)

    flow = Flow('LinearFlow2')
    runs = list(flow.runs())

    print(flow.latest_run)

    latest_run = flow.latest_run
    last_three_runs = runs[:3]
    print(last_three_runs)
    print(latest_run.id)
    print(latest_run.tags)
    print(latest_run.finished_at)
    print(latest_run.data)
    print(latest_run.data.fn_count)
    print(latest_run.data.title)
    print(latest_run.data.time_elapsed)

    run_list = [list_steps(run) for run in last_three_runs]

    get_runtime_comparisons(last_three_runs)

    get_convergence(last_three_runs)

    get_time_per_tolerance(last_three_runs)

def get_barchart(mf):
    flow = Flow('LinearFlow2')
    runs = list(flow.runs())
    latest_run = flow.latest_run
    steps = list(latest_run.steps())
    a = steps[2]
    print(a.task.data.orig)
    print(a.task.data.nlopt_weights)

    args_barchart = [{ 'y': a.task.data.orig, 'trace': "Original weights" },
                     { 'y': a.task.data.nlopt_weights, 'trace': "Computed weights" }]

    get_barplot(args_barchart, "big_1e-5.pdf", title="Average speedup of analytical gradients over finite differences")

    a = steps[6]
    print(a.task.data.orig)
    print(a.task.data.nlopt_weights)

    args_barchart = [{ 'y': a.task.data.orig, 'trace': "Original weights" },
                     { 'y': a.task.data.nlopt_weights, 'trace': "Computed weights" }]

    get_barplot(args_barchart, "animal_1e-5.pdf", title="Average speedup of analytical gradients over finite differences")

mf = Metaflow()
#list_flows(mf)
get_barchart(mf)
